In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os
tf.__version__
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# def clean_data(text):
#     text = text.lower()
#     text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
#     text = re.sub(r"i'm","i am",text)
#     text = re.sub(r"he's","he is",text)
#     text = re.sub(r"she's","she is",text)
#     text = re.sub(r"it's","it is",text)
#     text = re.sub(r"they're","they are",text)
#     text = re.sub(r"there're","there are",text)
#     text = re.sub(r"there's","there is",text)
#     text = re.sub(r"how're","how are",text)
#     text = re.sub(r"what're","what are",text)
#     text = re.sub(r"where're","where ",text)
#     text = re.sub(r"who're","who are",text)
#     text = re.sub(r"that're","that are",text)
#     text = re.sub(r"when're","when are",text)
#     text = re.sub(r"how's","how is",text)
#     text = re.sub(r"what's","what is",text)
#     text = re.sub(r"where's","where is",text)
#     text = re.sub(r"who's","who is",text)
#     text = re.sub(r"that's","that is",text)
#     text = re.sub(r"when's","when is",text)
#     text = re.sub(r"won't","would not",text)
#     text = re.sub(r"nt't","can not",text)
#     text = re.sub(r"\'bout'","about",text)
#     text = re.sub(r"\'till'","untill",text)
#     text = re.sub(r"\'ll","will",text)
#     text = re.sub(r"\'ve","have",text)
#     text = re.sub(r"\'re","are",text)
#     text = re.sub(r"\'d","would",text)
#     text = re.sub(r"\."," ",text)
#     text = re.sub(r"\,"," ",text)
#     text = re.sub(r"\!"," ",text)
#     text = re.sub(r"\?"," ",text)
#     text = re.sub(r"\;"," ",text)
#     text = re.sub(r"\:"," ",text)
#     return text

    
# df['text'] = df['text'].apply(clean_data)
# df['augmented_text'] = df['augmented_text'].apply(clean_data)
# df_val['text'] = df['text'].apply(clean_data)
# df_val['augmented_text'] = df['augmented_text'].apply(clean_data)



In [4]:
# data is being preprocessed, use it to save time for training
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Copy of data.csv')



In [5]:
max_length = 8

In [6]:

df = data[data['text'].apply(lambda x: len(x.split(" "))< max_length +1)]

In [8]:

train_df = df['augmented_text']
test_df = df['text']
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_df, test_df, test_size=0.2, random_state=24)
val_x, test_x,val_y, test_y = train_test_split(test_x, test_y, test_size = 0.5, random_state=24)


In [9]:
train_x

402452               elagl fight over alp cmapagin brochure
45026                 0sczrs qhoq goes on but no red dar0et
658341     solar hokd precibft launched in qbite gum valley
207624      apm housing rental eroprt darwin most expenisev
917206           us soldier klilde 4 uhtr in abgdhad attack
                                ...                        
577944                 airoptr sells benefits of new urcfew
623630             chxllehge for tasmanias housing inwusfry
254890    action group clasrifies menindee nwatecr situa...
1081         qarkizy criticises us yustic2 in polsnzki case
292316         oppoistoin questions power sattoin viability
Name: augmented_text, Length: 611100, dtype: object

In [10]:
val_x

423286    serial drink xiver allegedly caught drumk drkv...
574202    tourism bpry confident of improved faculties a...
147188                   btteer to come from norman  odhges
802320    reportevr baea5ts ppnolice to witness in backp...
151734    uji9n wants rethink on teacher crimijsl record...
                                ...                        
132413             phoenix expevtee to ta  fight to t bkrdq
57286     timothy baker not guklt5 of murveding glwdo mi...
396333                a few words with the lcubhosue ledare
704777                congo cu rigbleader surrounded kabila
301112        ex62nded interview qitg diuy ferguson ap golf
Name: augmented_text, Length: 76388, dtype: object

In [12]:

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(train_x)
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(train_x)
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding='post')


text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(train_y)
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(train_y)
test_padded = pad_sequences(test_sequences,maxlen = max_length, padding='post')





In [13]:
val_train_sequences = g_text_tokenizer.texts_to_sequences(val_x)
val_train_padded = pad_sequences(val_train_sequences, maxlen = max_length, padding='post')

val_test_sequences = text_tokenizer.texts_to_sequences(val_y)
val_test_padded = pad_sequences(val_test_sequences, maxlen = max_length, padding='post')

In [15]:
g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1
print(g_vocab_size, vocab_size)

844814 79061


In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=g_vocab_size, output_dim=128, input_length=max_length, mask_zero=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.RepeatVector(max_length),
    tf.keras.layers.LSTM(128,return_sequences=True),
    tf.keras.layers.LSTM(128,return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])
learning_rate = 0.005
optimizer = tf.keras.optimizers.RMSprop(learning_rate)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

In [18]:
model.summary(expand_nested=True,
    show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 embedding (Embedding)       (None, 8, 128)            10813619  Y          
                                                       2                    
                                                                            
 lstm (LSTM)                 (None, 128)               131584    Y          
                                                                            
 repeat_vector (RepeatVector  (None, 8, 128)           0         Y          
 )                                                                          
                                                                            
 lstm_1 (LSTM)               (None, 8, 128)            131584    Y          
                                                                            
 lstm_2 (LSTM)               (None, 8, 128)            1

In [19]:
train_padded.shape

(611100, 8)

In [20]:
temp_test_padded = test_padded.reshape((test_padded.shape[0], test_padded.shape[1],1))
temp_train_padded = train_padded.reshape((train_padded.shape[0], train_padded.shape[1],1))


In [21]:
epoch = 14
his = model.fit(train_padded, test_padded, batch_size=512, epochs=epoch, validation_data = (val_train_padded, val_test_padded))

Epoch 1/14


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 108136192 elements. This may consume a large amount of memory.
  num_elements)


1194/1194 [==============================] - 383s 313ms/step - loss: 5.5646 - accuracy: 0.3287 - val_loss: 6.7114 - val_accuracy: 0.3945
Epoch 2/14
1194/1194 [==============================] - 371s 311ms/step - loss: 3.5042 - accuracy: 0.5490 - val_loss: 6.8304 - val_accuracy: 0.5176
Epoch 3/14
1194/1194 [==============================] - 371s 311ms/step - loss: 2.7869 - accuracy: 0.6557 - val_loss: 7.1075 - val_accuracy: 0.5499
Epoch 4/14
1194/1194 [==============================] - 371s 311ms/step - loss: 2.3859 - accuracy: 0.7051 - val_loss: 7.8377 - val_accuracy: 0.5639
Epoch 5/14
1194/1194 [==============================] - 371s 311ms/step - loss: 2.1134 - accuracy: 0.7385 - val_loss: 9.0016 - val_accuracy: 0.5733
Epoch 6/14
1194/1194 [==============================] - 371s 311ms/step - loss: 1.8790 - accuracy: 0.7659 - val_loss: 11.5841 - val_accuracy: 0.5852
Epoch 7/14
1194/1194 [==============================] - 372s 311ms/step - loss: 1.6835 - accuracy: 0.7898 - val_loss: 13.0

In [34]:
# model.save('/content/drive/MyDrive/Colab_Notebooks/accuracy_84_3lstm_rnn_8inputlength')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/accuracy_84_3lstm_rnn_8inputlength/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/accuracy_84_3lstm_rnn_8inputlength/assets


In [ ]:
import matplotlib.pyplot as plt
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

In [35]:

test_sample = test_x.iloc[:10]
test_sample = g_text_tokenizer.texts_to_sequences(test_sample)
test_sample = pad_sequences(test_sample, maxlen=max_length, padding='post')

In [24]:
test_sample.shape

(2, 8)

In [25]:
test_x.iloc[:2]

810765    kroena war vetrenas lay wrethas at war memorial
647694           clmiaet change still on the radar survey
Name: augmented_text, dtype: object

In [26]:
test_y.iloc[:2]

810765    korean war veterans lay wreaths at war memorial
647694           climate change still on the radar survey
Name: text, dtype: object

In [36]:
pre = model.predict(test_sample)
pre.shape

(10, 8, 79061)

In [42]:
pre[0]

array([[3.0370647e-11, 9.9192306e-08, 1.5794976e-09, ..., 1.0274919e-11,
        9.5638419e-12, 1.6759474e-11],
       [2.0466103e-18, 6.9998867e-19, 5.4914064e-18, ..., 4.2240773e-20,
        3.9531659e-20, 2.0152629e-20],
       [2.4330707e-10, 7.1795951e-09, 1.5694761e-11, ..., 2.3098025e-13,
        1.8593799e-13, 1.1228942e-13],
       ...,
       [3.6076637e-16, 4.7104529e-16, 3.4065063e-16, ..., 1.3761941e-18,
        1.3570927e-18, 4.6012402e-19],
       [2.2333469e-16, 1.2614383e-19, 1.9610285e-17, ..., 7.9882277e-23,
        1.2472492e-22, 2.0075050e-22],
       [1.0000000e+00, 2.6886609e-32, 1.6186549e-27, ..., 1.1413105e-34,
        9.2468467e-35, 1.7932336e-34]], dtype=float32)

In [28]:
np.argmax(pre[0],1)

array([1295,  245, 1658, 3768,   11,  245,  881,    0])

In [29]:
def deTokenize(logits):
    index_to_words = {id: word for word, id in text_tokenizer.word_index.items()}
    index_to_words[0] = ''
    pre_index = np.argmax(logits, 1)
    return ' '.join(index_to_words[prediction] for prediction in pre_index)

In [47]:
print("Iutput:    ",test_x.iloc[1])
print("Actual:    ",test_y.iloc[1])
print("Predicted: ",deTokenize(pre[1]))

Iutput:     clmiaet change still on the radar survey
Actual:     climate change still on the radar survey
Predicted:  climate change still on the radar survey 


In [43]:
predict = []
for i in range(10):
  predict.append(deTokenize(pre[i]))



In [44]:
predict

['korean war veterans lay at war memorial ',
 'climate change still on the radar survey ',
 'form populations threatens naplan test potentially union says',
 'bargaining risk tasmania processor    ',
 'rhino charges promoted family at australia zoo ',
 'man wins retrial over sex   ',
 'health service mental health    ',
 '11 arrested travelling to 1000   ',
 'premier attacks opposition over foi   ',
 'cowboys coach courage nrl fine   ']

In [45]:
eva = pd.DataFrame({'input': test_x.iloc[:10], 'output': test_y.iloc[:10], 'predict':predict})

In [46]:
eva

,input,output,predict
810765,kroena war vetrenas lay wrethas at war memorial,korean war veterans lay wreaths at war memorial,korean war veterans lay at war memorial
647694,clmiaet change still on the radar survey,climate change still on the radar survey,climate change still on the radar survey
906447,ofmr flaw threatens naplan estt accuracy union...,form flaw threatens naplan test accuracy union...,form populations threatens naplan test potenti...
43169,0ppy mildew risk tazmanix processor,poppy mildew risk tasmania processor,bargaining risk tasmania processor
849703,rhino carhges iirwn family at asurtalia zoo,rhino charges irwin family at australia zoo,rhino charges promoted family at australia zoo
890223,man wins retrial ov3e sex offrnde vharyes,man wins retrial over sex offence charges,man wins retrial over sex
447621,health service addressgin mental haelht,health service addressing mental health,health service mental health
329081,11 arrested rtavelilng to bathruts 1000,11 arrested travelling to bathurst 1000,11 arrested travelling to 1000
151255,premier attacks opposition oc4r foi tequesfs,premier attacks opposition over foi requests,premier attacks opposition over foi
598551,owcboys coach queries nrl atmuo fine,cowboys coach queries nrl tamou fine,cowboys coach courage nrl fine
